In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

# Байесовские А/Б-тесты

Показана реализация А/Б-тестов. Рассмотрено использование байесовского моделирования для сравнения конверсий и средних. Дополнительно обсуждаются множественные сравнения и транзакционная выручка на пользователя.

https://stepik.org/course/249642/promo

- [О курсе](#О-курсе)
- [А/Б-тесты](#А/Б-тесты)
- [Реализация А/Б-тестов](#Реализация-А/Б-тестов)
- [Байесовское моделирование](#Байесовское-моделирование)
- [Конверсии](#Конверсии)
- [Средние](#Средние)
- [Транзакционная выручка на пользователя](#Транзакционная-выручка-на-пользователя)
- [Множественные сравнения](#Множественные-сравнения)
- [Связь с p-значениями](#Связь-с-p-значениями)

# О курсе

Показано использование байесовского моделирования для оценки А/Б-тестов.

После общей схемы экспериментов приведен пример реализации А/Б-теста. Рассмотрено использование байесовского моделирования для сравнения основных типов метрик - конверсий и средних. Также обсуждаются множественные сравнения и моделирование выручки на пользователя в транзакционных сервисах. 

Блокнот с примерами байесовской обработки на Гитхабе https://github.com/andrewbrdk/Bayesian-AB-Testing . Все видео есть на [Ютубе](https://www.youtube.com/playlist?list=PLqgtGAeapsOPpV0FqeXEpWosHBW8ZebYl). Код к реализации А/Б-тестов в репозитории https://github.com/andrewbrdk/AB-Testing-Implementation .


Рекомендации:  
Теория вероятностей:  
-J. Tsitsiklis, P. Jaillet, Introduction to Probability ([видео](https://www.youtube.com/playlist?list=PLUl4u3cNGP60hI9ATjSFgLZpbNJ7myAg6), [материалы](https://ocw.mit.edu/courses/res-6-012-introduction-to-probability-spring-2018/))  
Байесовское моделирование:  
-B. Lambert, A Student’s Guide to Bayesian Statistics ([книга](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [материалы](https://study.sagepub.com/lambert))  
-R. McElreath, Statistical Rethinking: A Bayesian Course with Examples in R and STAN ([книга](https://www.routledge.com/Statistical-Rethinking-A-Bayesian-Course-with-Examples-in-R-and-STAN/McElreath/p/book/9780367139919), [видео](https://www.youtube.com/playlist?list=PLDcUM9US4XdPz-KxHM4XHt7uUVGWWVSus), [материалы](https://github.com/rmcelreath/stat_rethinking_2024))    
Платформы А/Б-тестов:  
-[GrowthBook](https://www.growthbook.io/)  


# А/Б-тесты

### Напоминание, теорвер

Действия пользователей в продукте и целевые метрики можно описывать случайными величинами. Различия их распределений между группами А/Б-теста при прочих равных условиях (внешние факторы, состав аудитории) можно объяснять версиями продукта. Точные распределения неизвестны. Иногда их удается приблизить аналитическими моделями, но даже в этом случае остаются неизвестны точные параметры. В эксперименте собирается выборка из распределений. По выборке нужно построить оценки точных распределений, их свойств и выбрать лучший вариант.

Пусть $X_1,X_2,\dots,X_n$ - независимые одинаково распределенные случайные величины. При большом количестве данных по [закону больших чисел](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D1%87%D0%B8%D1%81%D0%B5%D0%BB#%D0%91%D0%BE%D1%80%D0%B5%D0%BB%D0%B5%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%B7%D0%B0%D0%BA%D0%BE%D0%BD_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D1%87%D0%B8%D1%81%D0%B5%D0%BB) распределение выборки близко точному распределению. [Выборочное среднее](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BE%D1%87%D0%BD%D0%BE%D0%B5_%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5) $\overline{X}_n$ приближает точное среднее $E[X]$, [выборочная дисперсия](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B4%D0%B8%D1%81%D0%BF%D0%B5%D1%80%D1%81%D0%B8%D1%8F) $\sigma_n^2$ - точную дисперсию $\sigma^2$ .

$$
\begin{split} 
\text{Выборочное среднее:} \quad &\overline{X}_n = \frac{1}{n} \sum\limits_{i=1}^n X_i 
\\ 
\text{Выборочная дисперсия:} \quad &\sigma^2_n = \frac{1}{n} \sum\limits_{i=1}^n \left(X_i - \overline{X}_n\right)^2 \end{split}
$$

О выборочном среднем можно думать не только как о среднем в одной выборке, но и как о случайной величине. По [линейности математического ожидания](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D0%B6%D0%B8%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5#%D0%A1%D0%B2%D0%BE%D0%B9%D1%81%D1%82%D0%B2%D0%B0_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B6%D0%B8%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F) среднее $\overline{X}_n$ совпадает со средним исходного распределения $E[\overline{X}_n] = E[X]$. Дисперсия $\overline{X}_n$ связана с дисперсией исходного распределения соотношением $\sigma^2_{\overline{X}_n} = \sigma^2/n$. Стандартное отклонение этого распределения $\sigma_{\overline{X}_n}$ называют [стандартной ошибкой среднего](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D0%BD%D0%B4%D0%B0%D1%80%D1%82%D0%BD%D0%B0%D1%8F_%D0%BE%D1%88%D0%B8%D0%B1%D0%BA%D0%B0). 

$$
E[\overline{X}_n] = E[X],
\quad
\sigma^2_{\overline{X}_n} = \frac{\sigma^2}{n}
$$


Случайную величину c двумя возможными значениями - 0 и 1 - называют [случайной величиной с распределением Бернулли](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%91%D0%B5%D1%80%D0%BD%D1%83%D0%BB%D0%BB%D0%B8). Вероятности выпадения единицы p достаточно для задания распределения. Среднее $E[X]=p$, дисперсия $\sigma^2=p(1−p)$.

$$
P(x) = \begin{cases} p, & x=1 \\ 1-p, & x=0 \end{cases} \\ E[x] = p, \quad \sigma^2 = p (1 - p)
$$

In [ ]:
import plotly.graph_objects as go

p_A = 0.2

values = [0, 1]
probs_A = [1 - p_A, p_A]
width = 0.2

fig = go.Figure()
fig.add_trace(go.Bar(x=values, y=probs_A, name=f"A, p={p_A}", width=width, marker_color='black'))
fig.update_layout(
    title="Распределение Бернулли",
    #xaxis_title="",
    yaxis_title="Вероятность",
    barmode="group",
    yaxis_range=[0, 1.3],
    bargap=0.6,
    template="plotly_white",
)

fig.show()
#fig.write_image("./stepik_Bern_dist.png", scale=2)

### Задача: выборки из 2 распределений Бернулли

Из двух распределений Бернулли делают выборки. Вероятность выпадения единицы в одном распределении $p_A=0.2$, в другом $p_B=0.25$. Отметьте верные утверждения.

In [ ]:
import plotly.graph_objects as go

p_A = 0.2
p_B = 0.25

values = [0, 1]
probs_A = [1 - p_A, p_A]
probs_B = [1 - p_B, p_B]
width = 0.2

fig = go.Figure()
fig.add_trace(go.Bar(x=values, y=probs_A, name=f"A, p={p_A}", width=width, marker_color='black', opacity=0.3))
fig.add_trace(go.Bar(x=values, y=probs_B, name=f"B, p={p_B}", width=width, marker_color='black'))
fig.update_layout(
    title="Распределения Бернулли",
    #xaxis_title="",
    yaxis_title="Вероятность",
    barmode="group",
    yaxis_range=[0, 1.5],
    bargap=0.6,
    template="plotly_white",
)

fig.show()
#fig.write_image("./stepik_2Bern_dist.png", scale=2)

*Задача использует закон больших чисел: по мере набора данных распределения в выборках приближаются к точным распределениям, оценки свойств по выборкам - к точным значениям. Случайные величины с распределением Бернулли будут применяться для моделирования конверсий в А/Б-тестах - те же закономерности будут в экспериментах. Здесь пока нет байесовского моделирования, но у корректных моделей должно быть такое же поведение с ростом объема данных.* 

--Среднее Б больше А.  
E[x] = p, p_B = 0.25, p_A = 0.2

--Дисперсия Б больше А.  
sigma^2= p * (1 - p), sigma^2_A= 0.16, sigma^2_B = 0.1875

--По мере набора данных доля единиц в выборках будет стремиться к нулю.  
По закону больших чисел распределение в выборке будет приближаться к точному распределению. Т.е. доли единиц будут стремиться к 0.2 для А и 0.25 для Б.

--По мере набора данных выборочные средние будут приближаться к точным средним.  
Выборочное среднее - состоятельная оценка точного среднего.

--По мере набора данных выборочные дисперсии будут стремиться к нулю.  
Выборочные дисперсии будут приближаться к точным дисперсиям.

--По мере набора данных стандартные ошибки средних будут стремиться к нулю.  
Стандартная ошибка среднего - отношение стандартного отклонения точного распределения к корню из размера выборки. Дисперсии постоянны, выборка растет - отношение стремится к нулю.

--По мере набора данных разность выборочных средних будет стремиться к нулю.   
Выборочные средние стремятся к точным средним. Разность выборочных средних будет стремиться к разности точных средних.

--По мере набора данных вероятность, что случайная точка из выборки Б больше случайной точки из A, будет стремиться к 1.
Точка из выборки Б больше А, если из Б выбрана 1, из А 0. Вероятность стремится к 0.25 * 0.8.

--По мере набора данных вероятность, что выборочное среднее Б больше выборочного среднего А, будет стремиться к 1.  
Средние распределений выборочных средних будут стремиться к точным средним; стандартные ошибки средних стремятся к нулю, т.е. распределения сужаются. Пересечение распределений будет уменьшаться. Вероятность, что точка из одного распределения больше точки из другого будет стремиться к 1.

# Реализация А/Б-тестов

### Окружение

Для запуска примеров используйте виртуальное окружение:

```
python -m venv pyvenv
source ./pyvenv/bin/activate
pip install flask aiohttp playwright
playwright install chromium
```

### Пример

Для корректного сравнения групп пользователь должен видеть только один вариант эксперимента. Пользователей нескольких вариантов необходимо убирать из анализа. Если их много - останавливать эксперимент и проверять реализацию.

Деление трафика делают по клиентским программам (клиентам) - браузер на настольном компьютере, браузере на телефоне, мобильное приложение. Каждая программа при обращении на сервер получает уникальную группу. Посетитель может использовать несколько программ и попасть в них в разные группы. Если в этих программах были авторизации, можно связать их между собой по общему аккаунту и добиться одинаковых групп. Если авторизаций не было, разные клиентсткие программы считают относящимися к разным пользователям.

В реализация а/б-теста ниже группа генерируется на бэкэнде `variant = random.choice(['A', 'B'])`. Вариант страницы определяется в зависимости от группы  `{% if variant == 'A' %} ... {% endif %}`. Группа записывается в куки `response.set_cookie`. При повторном заходе проверяется выданная группа `request.cookies.get('variant')`; если она уже есть, новая не присваивается.

```
from flask import Flask, render_template_string, request, make_response
import random

app = Flask(__name__)

TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>A/B Test</title>
</head>
<body>
    <h1>A/B Test</h1>
    {% if variant == 'A' %}
        <h3>Variant A</h3>
        <button onclick="console.log('Click A')">Click A</button>
    {% else %}
        <h3>Variant B</h3>
        <button onclick="console.log('Click B')">Click B</button>
    {% endif %}
</body>
</html>
'''

@app.route('/')
def index():
    variant = request.cookies.get('variant')
    if variant not in ['A', 'B']:
        variant = random.choice(['A', 'B'])
    response = make_response(render_template_string(TEMPLATE, variant=variant))
    response.set_cookie('variant', variant, max_age=60*60*24*30)
    return response

if __name__ == '__main__':
    app.run(debug=True)
```


Для запуска сохраните код в фаил `abtests.py`. Активируйте виртуальное окружение `source pyvenv/bin/activate`. Команда `python abtests.py` запустит веб-сервер:

```
> python abtests.py

 * Serving Flask app 'abtests'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 731-047-491
127.0.0.1 - - [27/Nov/2025 04:34:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 04:34:09] "GET / HTTP/1.1" 200 -
```

Откройте указанный адрес http://127.0.0.1:5000 в браузере

![abtest](https://ucarecdn.com/a7fad247-6312-4880-aa4b-9c2e36b97c31/)

Группа сохраняется в куках. Чтобы увидеть другую версию либо зайдите в режиме инкогнито, либо очистите куки и обновите страницу:

![abtest_clear](https://ucarecdn.com/92b0abfe-d963-437d-86b8-87fbfe75e020/)

### Скрипт заходов

Скрипт имитирует посещения сайта. Функция main запускает браузер `p.chromium.launch(headless=True)` и делает `N` заходов на страницу `[simulate_visit(browser) for i in range(N)]`. Действия на странице заданы `simulate_visit`. Скрипт сверяет текст в элементе `h3` со строками `"Variant A"` и `"Variant B"`. В первом случае была показана группа A, во втором - Б. Выводится подсчет показанных вариантов.

```
import asyncio
import argparse
from collections import Counter
from playwright.async_api import async_playwright

BASE_URL = "http://127.0.0.1:5000"

MAX_CONCURRENT = 30
SEM = asyncio.Semaphore(MAX_CONCURRENT)

async def simulate_visit(browser):
    async with SEM:
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(BASE_URL)
        group = None
        await page.wait_for_selector("h1")
        h3 = await page.query_selector("h3")
        if h3:
            t = await h3.text_content()
            if "Variant A" in t:
                group = "A"
            elif "Variant B" in t:
                group = "B"
        await page.close()
        await context.close()
        return group

async def main():
    parser = argparse.ArgumentParser(description="Simulate A/B test visits")
    parser.add_argument(
        "-n", "--num-visits", type=int, default=1000,
        help="Number of visits to simulate (default: 1000)"
    )
    args = parser.parse_args()
    N = args.num_visits
    counts = Counter()
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        t = [simulate_visit(browser) for i in range(N)]
        results = await asyncio.gather(*t)
        for g in results:
            counts[g] += 1
        await browser.close()
    print("Exp Split:")
    for group in sorted(counts):
        part = (counts[group] / N) * 100
        print(f"{group}: {counts[group]} visits ({part:.2f}%)")
    
if __name__ == "__main__":
    asyncio.run(main())
```

Сохраните скрипт в фаил `simulate_visits.py`. Активируйте виртуальное окружение. Запустите сервер `python abtests.py`. При запущенном сервере в другой консоли выполните скрипт `python simulate_visits.py -n 100`. Опция `-n` задает количество заходов. В отладочном режиме сервер логирует поступающие запросы ( `"GET / HTTP/1.1" 200` ).

```
> python abtests.py 

 * Serving Flask app 'abtests'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 731-047-491
127.0.0.1 - - [27/Nov/2025 05:48:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 05:48:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 05:48:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 05:48:06] "GET / HTTP/1.1" 200 -
```

Скрипт `simulate_visits.py` выведет количество пользователей в каждой группе.


```
> python simulate_visits.py -n 1000

Exp Split:
A: 484 visits (48.40%)
B: 516 visits (51.60%)

```

Деление трафика близко ожидаемому 50/50.

### Задача - запись группы в куки

Что будет, если в функции index() не записывать группу эксперимента в куки? Отметьте верные утверждения.

```
@app.route('/')
def index():
    variant = request.cookies.get('variant')
    if variant not in ['A', 'B']:
        variant = random.choice(['A', 'B'])
    response = make_response(render_template_string(TEMPLATE, variant=variant))
    # response.set_cookie('variant', variant, max_age=60*60*24*30)
    return response
```

*Хранить и передавать группу именно в куках не обязательно. Важно, чтобы группа пользователя не менялась в 
течение эксперимента. Если меняется - это некорректно.*


-Пользователю будет присваиваться новая группа при каждом заходе на страницу  
-Группа пользователя будет постоянна все время эксперимента (если пользователь не затирает куки)  
-Эксперимент будет реализован корректно  
-Эксперимент будет реализован некорректно   

### Хэширование

Вместо вызова random группы могут определять хэшированием идентификатора клиента с названием эксперимента. В таком случае для равномерного деления по группам `group = hash(device_id || experiment) % n` , где `group` - группа, `device_id || experiment` - объединение идентификатора клиента и названия эксперимента, `hash` - хэш-функция,  `% n` - деление по модулю на количество групп.

Идентификатор клиента присваивается при первом обращении на сервер, сохраняется на клиенте и не меняется.

В хэширование добавляют название эксперимента для независимого распределения групп в разных экспериментах. Название должно быть уникальным. Для `random` нужно поддерживать отдельные генераторы случайных чисел на каждый эксперимент.

При хэшировании вычисление группы можно перенести на клиент. Клиент получает с сервера список активных экспериментов и для каждого вычисляет группу со своим `device_id`. Сервер все равно понадобится как минимум для управления экспериментами и поддержания одинаковых групп на разных устройствах.

Для двух равновероятных групп вместо деления по модулю достаточно смотреть на произвольный бит хэша. Ниже используется наименее значимый бит последнего байта `hash_bytes[-1] & 1`.


```
import uuid
import hashlib
from collections import Counter

def assign_group(device_id, experiment):
    key = f"{device_id}:{experiment}"
    hash_bytes = hashlib.sha256(key.encode()).digest()
    last_bit = hash_bytes[-1] & 1
    return 'A' if last_bit == 0 else 'B'

device_id = str(uuid.uuid4())
experiment = 'first_exp'
group = assign_group(device_id, experiment)
print(f"experiment: {experiment}, device_id: {device_id}, group: {group}")

N = 100000
groups = []
for _ in range(N):
    device_id = str(uuid.uuid4())
    groups.append(assign_group(device_id, experiment))
distribution = Counter(groups)
print(distribution)
```

---

```
> python hashing.py
 
experiment: first_exp, device_id: 08373255-45ed-4424-97b6-2c8f038c7f59, group: B
Counter({'A': 50045, 'B': 49955})
```

In [ ]:
import uuid
import hashlib
from collections import Counter

def assign_group(device_id, experiment):
    key = f"{device_id}:{experiment}"
    hash_bytes = hashlib.sha256(key.encode()).digest()
    b = hash_bytes[-1] & 1
    return 'A' if b == 0 else 'B'

device_id = '111'
experiment = 'first_exp'
group = assign_group(device_id, experiment)
print(f"experiment: {experiment}, device_id: {device_id}, group: {group}")

N = 100000
groups = []
for _ in range(N):
    device_id = str(uuid.uuid4())
    groups.append(assign_group(device_id, experiment))
distribution = Counter(groups)
print(distribution)

### Хэширование, задача

В какую группу попадет device_id = "123" в эксперименте experiment = "hashing" при использовании функции assign_group?

```
import hashlib

def assign_group(device_id, experiment):
    key = f"{device_id}:{experiment}"
    hash_bytes = hashlib.sha256(key.encode()).digest()
    b = hash_bytes[-1] & 1
    return 'A' if b == 0 else 'B'
```

В ответе приведите вычисленный хэш в шестнадцатеричном представлении и группу через пробел, например `8fadb70ca9367f3e4aacbefb85f3949679b29971cd7ccf4e17801de49c42de70 A` .  

In [ ]:
import hashlib

def assign_group(device_id, experiment):
    key = f"{device_id}:{experiment}"
    hash_bytes = hashlib.sha256(key.encode()).digest()
    last_bit = hash_bytes[-1] & 1
    group = 'A' if last_bit == 0 else 'B'
    return hash_bytes, group

device_id = "123"
experiment = "hashing"
hash_bytes, group = assign_group(device_id, experiment)
print(hash_bytes.hex(), group)

print('last', hex(hash_bytes[-1]), f'{hash_bytes[-1]:08b}', bin(hash_bytes[-1] & 1))
print('first', hex(hash_bytes[0]), f'{hash_bytes[0]:08b}', bin(hash_bytes[0] & 1))

In [ ]:
hex(hash_bytes[-1]), bin(hash_bytes[-1])


### Второй пример - события

Для оценки конверсии в нажатие кнопки реализована отправка событий просмотра страницы pageview и клика кнопки `button_click`. На бэкэнде добавлен эндпоинт `/events`. Метод `POST` записывает события в список `EVENTS`. В реальной конфигурации для сбора событий используют отдельный сервис. Метод `GET` загружает все полученные события. На клиенте добавлена функция отправки событий `sendEvent`. В каждом событии передается момент отправки `ts`, идентификатор клиента `device_id`, название события `event`, дополнительные параметры `params`. Также передается экспериментальная группа `variant`.

Событие захода на страницу отправляется при загрузке страницы `pageview`. При клике на кнопку отправляется событие `button_click`.  

```
import uuid
import random
from flask import Flask, request, make_response, render_template_string, jsonify

app = Flask(__name__)

TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>A/B Test</title>
</head>
<body>
    <h1>A/B Test</h1>
    {% if variant == 'A' %}
        <h3>Variant A</h3>
        <button onclick="sendEvent('button_click', { btn_type: 'A' })">Click A</button>
    {% else %}
        <h3>Variant B</h3>
        <button onclick="sendEvent('button_click', { btn_type: 'B' })">Click B</button>
    {% endif %}

    <script>
        async function sendEvent(eventName, params = {}) {
            let ts = new Date().toISOString();
            await fetch('/events', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    ts: ts,
                    device_id: deviceId,
                    event: eventName,
                    variant: variant,
                    params: params
                })
            });
        }

        function getCookie(name) {
            const value = `; ${document.cookie}`;
            const parts = value.split(`; ${name}=`);
            if (parts.length === 2) return parts.pop().split(';').shift();
        }

        const deviceId = getCookie("device_id");
        const variant = getCookie("variant");

        sendEvent("pageview", {});
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    device_id = request.cookies.get("device_id")
    if not device_id:
        device_id = str(uuid.uuid4())
    variant = request.cookies.get('variant')
    if variant not in ['A', 'B']:
        variant = random.choice(['A', 'B'])
    response = make_response(render_template_string(TEMPLATE, variant=variant))
    response.set_cookie('device_id', device_id)
    response.set_cookie('variant', variant, max_age=60*60*24*30)
    return response

EVENTS = []

@app.route('/events', methods=['GET', 'POST'])
def events():
    if request.method == 'POST':
        data = request.json
        EVENTS.append(data)
        return jsonify({"status": "ok"})
    else:
        return jsonify(EVENTS)

if __name__ == '__main__':
    app.run(debug=True)
```

Сохраните скрипт в фаил `abtests_events.py`, запустите в виртуальном окружении, откройте страницу в браузере. В логах запросов `POST /events` - получение событий.

```
> python abtests_events.py 

 * Serving Flask app 'abtests_events'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 751-617-936
127.0.0.1 - - [28/Nov/2025 04:55:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2025 04:55:51] "POST /events HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2025 04:56:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2025 04:56:04] "POST /events HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2025 04:56:08] "POST /events HTTP/1.1" 200 -
```

События видны в разделе Network в браузере.

![events_network](https://ucarecdn.stepik.net/4a9a64c2-a514-4f57-b012-0c67139aeb74/)

Эндпоинт `/events` отображает все пришедшие события.

![events](https://ucarecdn.stepik.net/b8eb3cab-b15e-4860-aa6d-aa8b2192a0b7/)

### Скрипт заходов, события

Скрипт имитирует заходы и клики. В `simulate_visit` добавлен вызов `page.click("button")` для нажатия кнопки на странице. Вероятность клика в группах разная и задается `CLICK_PROBS`. При заходе на страницу и при нажатии кнопки на сервер отправляются события `pageview` и `button_click`. По этим событиям в скрипте считается конверсия в клик и сравнивается с заданной `CLICK_PROB`. Функция `fetch_events` получает с сервера все собранные события, `count_visits_clicks` считает общее количество показов и кликов в разбивке по группам и конверсию в клик. 

```
import random
import aiohttp
import asyncio
import argparse
from collections import Counter
from playwright.async_api import async_playwright

BASE_URL = "http://127.0.0.1:5000"
CLICK_PROBS = {
    'A': 0.1,
    'B': 0.2
}
MAX_CONCURRENT = 30
SEM = asyncio.Semaphore(MAX_CONCURRENT)

async def simulate_visit(browser):
    async with SEM:
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(BASE_URL)
        group = None
        await page.wait_for_selector("h1")
        h3 = await page.query_selector("h3")
        if h3:
            t = await h3.text_content()
            if "Variant A" in t:
                group = "A"
            elif "Variant B" in t:
                group = "B"
        if random.random() < CLICK_PROBS.get(group):
            await page.click("button")
        await page.close()
        await context.close()
        return group

async def fetch_events():
    url = f"{BASE_URL}/events"
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, timeout=5) as resp:
                resp.raise_for_status()
                return await resp.json()
    except Exception as e:
        return None

async def count_visits_clicks():
    events = await fetch_events()
    if events is None:
        return None
    device_variants = {}
    for e in events:
        d = e["device_id"]
        v = e["variant"]
        device_variants.setdefault(d, set()).add(v)
    conflicted = {d for d, vs in device_variants.items() if len(vs) > 1}
    if len(conflicted) > 0:
        print("Multiple variants:\n", conflicted)
    else:
        print("No device_ids with multiple variants found")
    views = {}
    clicks = {}
    for e in events:
        d = e["device_id"]
        if d in conflicted:
            continue
        v = e["variant"]
        evt = e["event"]
        if evt == "pageview":
            views[v] = views.get(v, 0) + 1
        elif evt == "button_click":
            clicks[v] = clicks.get(v, 0) + 1
    summary = []
    variants = set(list(views.keys()) + list(clicks.keys()))
    for v in variants:
        summary.append({
            "variant": v,
            "views": views.get(v, 0),
            "clicks": clicks.get(v, 0)
        })
    return summary


async def main():
    parser = argparse.ArgumentParser(description="Simulate A/B test visits")
    parser.add_argument(
        "-n", "--num-visits", type=int, default=1000,
        help="Number of visits to simulate (default: 1000)"
    )
    args = parser.parse_args()
    N = args.num_visits
    counts = Counter()
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        t = [simulate_visit(browser) for i in range(N)]
        results = await asyncio.gather(*t)
        for g in results:
            counts[g] += 1
        await browser.close()
    print("Exp Split:")
    for group in sorted(counts):
        part = (counts[group] / N) * 100
        print(f"{group}: {counts[group]} visits ({part:.2f}%)")
    print("")
    print("Exp events:")
    summary = await count_visits_clicks()
    summary.sort(key=lambda x: x['variant'])
    for s in summary:
        g, v, c = s['variant'], s['views'], s['clicks']
        print(f"Group {g}: views {v}, clicks {c} (CTR {c/v*100:.2f}%, CLICK_PROBS {CLICK_PROBS[g]*100:.2f}%)")
    print("")
    
if __name__ == "__main__":
    asyncio.run(main())
```

Сохраните код в фаил `simulate_visits_events.py`. Запустите `abtest_events.py` предыдущего шага.

```
> python abtests_events.py 

 * Serving Flask app 'abtests_events'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 385-438-327
127.0.0.1 - - [27/Nov/2025 15:14:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 15:14:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 15:14:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 15:14:13] "POST /events HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 15:14:13] "POST /events HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2025 15:14:13] "GET / HTTP/1.1" 200 -
```

При запущенном сервере выполните `simulate_visits_events.py`. Доля пользователей в группах и конверсии в клик близки ожидаемым.

```
> python simulate_visits_events.py -n 1000

Exp Split:
A: 498 visits (49.80%)
B: 502 visits (50.20%)

Exp events:
No device_ids with multiple variants found
Group A: views 498, clicks 50 (CTR 10.04%, CLICK_PROBS 10.00%)
Group B: views 502, clicks 86 (CTR 17.13%, CLICK_PROBS 20.00%)
```

### Задача: добавить группу в тест


Добавьте еще одну группу в эксперимент и проверьте деление трафика и конверсию в клик.

- В блок `{% if variant == 'A' %} ... {% endif %}` на сервере добавьте группу `C`. Содержание сделайте аналогичным предыдущими вариантами.
- Убедитесь, что функция `index()` на сервере корректно делит по группам. Для выдачи групп используйте `random.choice(...)` с 3 вариантами.
- В скрипте заходов добавьте вероятность клика по кнопке в группе `C` в `CLICK_PROBS` равную 0.3 .
- В функции `simulate_visit` добавьте определение группы `C` по содержанию элемента `h3`.  

Для ответа зафиксируйте зерно случайного генератора на сервере `random.seed(1)`. Запустите сервер в однопоточном режиме (как в примерах). Выполните 1000 запросов на сервер. В ответе приведите показы групп A, B и C через пробел: `321 332 347`.

### Задача: произвольные доли групп

В предыдущих примерах пользователи делились между группами поровну. Это необязательно - соотношение трафика между группами может быть любым, в том числе меняться по ходу эксперимента.

Реализуйте функцию assign_group, принимающую на вход массив вероятностей групп и возвращающую номер группы в соответствии с заданными вероятностями. Для принятия ответа функция должна проходить проверку ниже:

```
from math import sqrt

def check(probs):
    N = 100_000
    counts = [0] * len(probs)
    for _ in range(N):
        g = assign_group(probs)
        counts[g] += 1
    for c, p in zip(counts, probs):
        samp_p = c / N
        stderr = sqrt(p * (1-p) / N)
        if samp_p < p - 5 * stderr or samp_p > p + 5 * stderr:
            return False
    return True
```

### Проверка

**Редактор** 

```
::python3.12

::code
import random

def assign_group(probs: list[float]) -> int:
    # your code here

::footer
def gen_n_groups(probs, N=100_000):
    counts = [0] * len(probs)
    for _ in range(N):
        g = assign_group(probs)
        counts[g] += 1
    return " ".join(str(g) for g in counts)

probs = [float(x.strip()) for x in input().split()]
print(gen_n_groups(probs))
```

**Проверки**

```
# This is a sample Code Challenge
# Learn more: https://stepik.org/lesson/9172
# Ask your questions via help@stepik.org

def generate():
    w = [
        (0.5, 0.5),
        (0.9, 0.1),
        (0.3, 0.5, 0.2),
        (0.25, 0.25, 0.3, 0.2),
        (0.4, 0.2, 0.15, 0.15, 0.1)
    ]
    tests=[(" ".join(str(i) for i in x) + "\n", x) for x in w] 
    return tests

def check(reply, clue):
    from math import sqrt
    N = 100_000
    probs = clue
    counts = [int(x.strip()) for x in reply.split()]
    for c, p in zip(counts, probs):
        samp_p = c / N
        stderr = sqrt(p * (1-p) / N)
        if samp_p < p - 5 * stderr or samp_p > p + 5 * stderr:
            return False
    return True

# def solve(dataset):
#     a, b = dataset.split()
#     return str(int(a) + int(b))
```

In [ ]:
#1
import random

def assign_group(probs: list[float]) -> int:
    r = random.random()
    cumulative = 0.0
    for i, p in enumerate(probs):
        cumulative += p
        if r < cumulative:
            return i
    return len(probs) - 1

#2 
import random

def assign_group(probs: list[float]) -> int:
    return random.choices(range(len(probs)), probs)[0]

# Байесовское моделирование

### Соотношение Байеса

В байесовском моделировании для объяснения данных/наблюдений/фактов $\mathcal{D}$ предлагают гипотезы/модели $\mathcal{H}_i$. Вычисляют вероятности получить данные в рамках каждой модели - функции правдоподобия $P(\mathcal{D}|\mathcal{H}_i)$. Прошлый опыт учитывают уверенностью в модели - априорных вероятностях $P(\mathcal{H}_i)$. По соотношению Байеса вычисляют обновленную уверенность с учетом данных - апостериорные вероятности $P(\mathcal{H}_i|\mathcal{D})$. На их основе выбирают наиболее подходящую модель. Модели необходимо валидировать - хотя для объяснения данных удается выбрать лучшую из предложенных гипотез, ни одна из гипотез может не соответствовать реальности.

$$
\begin{split}
P(\mathcal{H}_i | \mathcal{D}) &= \frac{ P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }{P(\mathcal{D})}
= \frac{ P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }{\sum \limits_i P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i) }
\\
P(\mathcal{H}_i | \mathcal{D}) &\mbox{ - апостериорное распределение вероятности} 
\\
P(\mathcal{D} | \mathcal{H}_i) &\mbox{ - функция правдоподобия}
\\
P(\mathcal{H}_i) &\mbox{ - априорное распределение вероятности}
\\
P(\mathcal{D}) &\mbox{ - нормировочный коэффициент}
\end{split}
$$

&nbsp;

<center>
<img src="../figs/bayes_hypotheses_square.png" alt="bayes_hypotheses_square" width="900"/>
</center>

### Задача: выбор гипотез

На дашборде видно падение конверсии в оплату по сравнению с предыдущими днями. Возможные причины падения $H$ и оценки их априорных вероятностей по прошлому опыту $P(H)$ приведены в таблице ниже. С учетом детальной информации $D$ - величины падения, поведения других метрик, времени релизов - сделана оценка правдоподобия каждого варианта $P(D|H)$. Посчитайте вероятности причин падения метрики. В ответе укажите наибольшую вероятность, переведенную в проценты и округленную до целых.


| Гипотеза H | Априорная вероятность P(H) | Правдоподобие P(D\|H) |
|------------|------------|----------------------|
| Ошибка отчетов | 0.50 | 0.10 |
| Баг на проде | 0.25 | 0.20 |
| Проблемы в отдельных сегментах | 0.20 | 0.70 |
| Другие причины | 0.05 | 0.80 |



Задача показывает расчет апостериорных вероятностей конечного количества гипотез. На практике не очевидно, как задавать априорные вероятности и правдоподобия. Формально их можно оценить, если вести журнал инцидентов - для каждого отклонения фиксировать набор "симптомов" и итоговую установленную причину. По этим данным можно оценить частоту причин P(H) и "симптомов" в рамках каждой причины P(D|H).

In [ ]:
prior = [0.5, 0.25, 0.2, 0.05]
like = [0.1, 0.2, 0.7, 0.8]
post = [p * l for p, l in zip(prior, like)]
norm = sum(post)
post = [p / norm for p in post]
post

### Задача: базовый процент

Подозрительных транзакций 2%.  
Они помечаются с вероятностью 99%.  
Обычные транзакции помечаются подозрительными с вероятностью 3%.  
Сколько среди отмеченных транзакций обычных?  
Ответ округлите до целых. 

Это стандартная задача на соотношение Байеса. Конкертные вероятности напоминают об ошибке базового процента https://en.wikipedia.org/wiki/Base_rate_fallacy . Хотя вероятность пометить подозрительную транзакцию 99%, среди всех помеченных транзакций реально подозрительных меньше половины, остальные - ложно отмеченные обычные.  Базовый процент учитывается априорной вероятностью. Его также важно учитывать при сравнении гипотез в байесовском моделировании.

In [ ]:
#           | susp, 2% | common
# check, %  | 99       |  3
# no check  |          |  

p_susp = 0.02
p_susp_check = 0.99
p_com_check = 0.03
p_com = 1 - p_susp

#p_com * p_com_check / (p_susp * p_susp_check + p_com * p_com_check)
p_susp * p_susp_check / (p_susp * p_susp_check + p_com * p_com_check)

Ботов в трафике 3%.  
Они распознаются с вероятностью 99%.  
Реальные пользователи помечаются как боты с вероятностью 4%.  
Сколько среди отмеченных ботами реальных пользователей?  
Ответ округлите до целых. 

In [ ]:
#           | bot, 3% | real, 97%
# marked    | 99      |  4
# nonmarked |         |  

p_bot = 0.03
p_bot_mark = 0.99
p_real = 1 - p_bot
p_real_mark = 0.04

p_real * p_real_mark / (p_bot * p_bot_mark + p_real * p_real_mark)

### Задача: качество гипотез

Кроме вероятности выбрать лучшую группу в экспериментах важно качество гипотез.

Изменение может улучшить продукт с вероятностью $h$ или ухудшить с вероятностью $1-h$.  
Вы распознаете лучшую группу в эксперименте с вероятностью $p$.  
Всего провели $N$ экспериментов.  
Посчитайте количество внедренных улучшений и ухудшений для $N=50, p = 95\%, h = 10\%$.  
В ответе запишите количество улучшений и ухудшений через пробел, округлите до целых.

<center>
<img src="../figs/bayes_better_worse_guess.png" alt="bayes_better_worse_guess" width="500"/>
</center>

In [ ]:
N = 50
p = 0.95
h = 0.1

S = N * h * p
print('S', S)

L = N * (1-h) * (1-p)
print('L', L)

Улучшения определяются качеством гипотез.  
Можно добиться такого же количества улучшений с меньшим количеством экспериментов, но с большим качеством.

In [ ]:
N = 10
p = 0.95
h = 0.3

S = N * h * p
print('S', S)

L = N * (1-h) * (1-p)
print('L', L)

# Конверсии

### Модель

Конверсии можно моделировать последовательностью случайных величин с двумя возможными исходами - успехом и неудачей. Для оценки вероятности успеха $p$ в серии $N$ испытаний с $n_s$ успехами правдоподобие удобно задавать биномиальным распределением $P(\mathcal{D} | \mathcal{H}) = \mbox{Binom}(n_s, N | p)$, априорное распределение - бета-распределением $P(\mathcal{H}) = \mbox{Beta}(p; \alpha, \beta)$ с параметрами $\alpha, \beta$. Тогда апостериорная вероятность также будет бета-распределением с обновленными параметрами $P(\mathcal{H} | \mathcal{D}) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)$. При достаточно большом количестве данных $N \gg n_s \gg \alpha, \beta$ бета-распределение близко нормальному $\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) \approx \mbox{Norm}(x; \mu, \sigma^2), \, \mu = n_s / N, \, \sigma^2 = \mu (1 - \mu) / N$.

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p)
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta)
$$

$$
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
$$


$$
N \gg n_s \gg \alpha, \beta: \quad
\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) 
\approx \mbox{Norm}(x; \mu, \sigma^2),
\quad
\mu = n_s / N, 
\,
\sigma^2 = \mu (1 - \mu) / N
$$


$$
\mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\qquad
\mbox{Beta}(x; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\qquad
\mbox{Norm}(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
$$

### Задача 1, абсолютная разность

На версию А страницы веб-сайта зашло $N_A=10000$ человек, $n_{s_A}=1000$ нажали кнопку "Продолжить". На версию Б также зашло $N_B=10000$ человек, кнопку продолжить нажали $n_{s_B}=1100$. С какой вероятностью конверсия страницы Б выше страницы А на 1 процентный пункт $P(p_B−p_A>0.01)$? Вероятность переведите в проценты и округлите до целых.

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

d = 0.01

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(d)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b - samp_a > d) / npost

print(f"P(p_b - p_a > {d}) diff dist: {approx_diff_p}")
print(f"P(p_b - p_a > {d}) post samples: {samp_diff_p}")

# P(p_b - p_a > 0.01) diff dist: 0.49981600299255935
# P(p_b - p_a > 0.01) post samples: 0.50065

### Относительная разность, распространение ошибок

В результатах удобна относительная разность

$$
\Delta p = \frac{p_B - p_A}{p_A}
$$

В общем случае распределение $\Delta p$ задается [сверткой](https://en.wikipedia.org/wiki/Ratio_distribution) плотностей вероятности $p_A, p_B$. Приближенно если распределения $p_A, p_B$ нормальные, нет ковариации $Cov(p_A, p_B)=0$, плотность вероятности $p_A$ мала вблизи 0, то распределение $\Delta p$ также можно считать нормальным - см. [распространение ошибок](https://en.wikipedia.org/wiki/Propagation_of_uncertainty) и [дельта-метод](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D0%BB%D1%8C%D1%82%D0%B0-%D0%BC%D0%B5%D1%82%D0%BE%D0%B4_(%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0)):

$$
P_{p_A}(x) = \text{Norm}(x; \mu_A, \sigma^2_A), \quad P_{p_B}(x) = \text{Norm}(x; \mu_B, \sigma^2_B) \\ P_{\Delta p}(x) \approx \text{Norm}(x; \mu_{\Delta}, \sigma_{\Delta}^2), \quad \mu_{\Delta} = \frac{\mu_B - \mu_A}{\mu_A}, \quad \sigma_{\Delta} = \frac{|\mu_B|}{|\mu_A|} \sqrt{ \frac{\sigma_A^{2}}{\mu_A^{2}} + \frac{\sigma_B^{2}}{\mu_B^{2}} }
$$

Распределение относительной разности можно оценить численно сэмплированием апостериорных распределений $p_A, p_B$. На левом графике ниже задано два исходных нормальных распределения. На правом - плотность вероятности относительной разности по приближенному соотношению и с помощью сэмплирования визуально близки.

<div style="display: flex; gap: 10px;">
    <img src="https://ucarecdn.stepik.net/1b72155f-72f6-4034-873b-c6d7e04f5764/" width="40%"/>
    <img src="https://ucarecdn.stepik.net/a065d74a-8124-4683-952d-83fa1007be40/" width="40%"/>
</div>

Приближение будет применимо хуже, если у $p_A$ есть значимая плотность вероятности вблизи 0. На практике с этим можно встретиться при оценке выручки на пользователя с учетом неплатящих пользователей. На левом графике ниже нормальные распределения заданы ближе к нулю, чем в предыдущем случае. На правом видно более сильное отличие приближенного распределения относительной разности от построенного сэмплированием.

<div style="display: flex; gap: 10px;">
    <img src="https://ucarecdn.stepik.net/a36d2258-dc38-4c81-8349-790c979d6778/" width="40%"/>
    <img src="https://ucarecdn.stepik.net/9a985d79-cc7f-4a12-9708-a553749192ac/" width="40%"/>
</div>

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

mua = 1
sa = 0.1
mub = 1.1
sb = 0.1
mur = mub / mua - 1
sr = np.abs(mub / mua) * np.sqrt( (sa/mua)**2 + (sb/mub)**2 )

da = stats.norm(loc=mua, scale=sa)
db = stats.norm(loc=mub, scale=sb)
dr = stats.norm(loc=mur, scale=sr)

x = np.linspace(-1, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=da.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='А'))
fig.add_trace(go.Scatter(x=x, y=db.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Б', opacity=0.3))
fig.add_vline(0, line_dash='dash', line_color='black')
fig.update_layout(title='Распределения А, Б',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[-0.5, 3])
fig.show()

N = 100000
samp_a = da.rvs(N)
samp_b = db.rvs(N)
samp_r = (samp_b - samp_a) / samp_a

x = np.linspace(-3, 3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=dr.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Распространение ошибок'))
fig.add_trace(go.Histogram(x=samp_r, histnorm='probability density', name='Сэмплирование', nbinsx=1000,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Относительная разность',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[-3, 3])
fig.show()

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

mua = 0.1
sa = 0.07
mub = 0.13
sb = 0.07
mur = mub / mua - 1
sr = np.abs(mub / mua) * np.sqrt( (sa/mua)**2 + (sb/mub)**2 )

da = stats.norm(loc=mua, scale=sa)
db = stats.norm(loc=mub, scale=sb)
dr = stats.norm(loc=mur, scale=sr)

x = np.linspace(-1, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=da.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='А'))
fig.add_trace(go.Scatter(x=x, y=db.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Б', opacity=0.3))
fig.add_vline(0, line_dash='dash', line_color='black')
fig.update_layout(title='Распределения А, Б',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[-1, 1])
fig.show()

N = 100000
samp_a = da.rvs(N)
samp_b = db.rvs(N)
samp_r = (samp_b - samp_a) / samp_a

x = np.linspace(-10, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=dr.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Распространение ошибок'))
fig.add_trace(go.Histogram(x=samp_r[(samp_r > -10) & (samp_r < 10)], histnorm='probability density', name='Сэмплирование', nbinsx=1000,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Относительная разность',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[-7, 10])
fig.show()

### Задача 2, относительная разность

На версию А страницы веб-сайта зашло $N_A=10000$ человек, $n_{s_A}=1000$ нажали кнопку "Продолжить". На версию Б также зашло $N_B=10000$ человек, кнопку продолжить нажали $n_{s_B}=1100$. С какой вероятностью конверсия страницы Б выше страницы А на 1 процент $P(\Delta p > 0.01)$? Вероятность переведите в проценты и округлите до целых.

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

rd = 0.01

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

rd_mu = (p_dist_b.mean() - p_dist_a.mean()) / p_dist_a.mean()
rd_s = np.abs(p_dist_b.mean() / p_dist_a.mean()) * np.sqrt((p_dist_a.std() / p_dist_a.mean())**2 + (p_dist_b.std() / p_dist_b.mean())**2)

approx_reldiff_dist = stats.norm(loc=rd_mu, scale=rd_s)
approx_reldiff_p = 1 - approx_reldiff_dist.cdf(rd)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_reldiff_p = np.sum((samp_b - samp_a)/ samp_a > rd) / npost

print(f"P((p_b-p_a)/p_a > {d}) rel diff dist: {approx_reldiff_p}")
print(f"P((p_b-p_a)/p_a > {d}) post samples: {samp_reldiff_p}")

# P((p_b-p_a)/p_a > 0.01) diff dist: 0.97604762812634
# P((p_b-p_a)/p_a > 0.01) post samples: 0.98068

### Задача 3, априорные для конверсий

Априорное распределение можно задавать по историческим данным. Метрики эксперимента будут близки метрикам до эксперимента. Поэтому вместо равномерного от 0 до 1 априорного распределения $P(p)= \mbox{Beta}(p;\alpha=1, \beta=1)=1$ параметры $\alpha,\beta$ можно подобрать для соответствия априорного распределения исторической конверсии. В апостериорное распределение они входят в виде $P(p∣N,n_s)= \mbox{Beta}(p;\alpha+n_s, \beta+N−n_s)$. Поэтому точное значение параметров важно пока данных мало, но при большом количестве данных $N,N−n_s \gg \alpha, \beta$ их выбор перестает играть решающую роль.

На версию А страницы сайта зашло $N_A=10000$ человек, $n_{s_A}=1000$ нажали кнопку "Продолжить". На версию Б также зашло $N_B=10000$ человек, кнопку нажали $n_{s_B}=1100$. До эксперимента на сайт зашло 50000 человек, кнопку "Продолжить" нажали 5200. Посчитайте вероятность $P(p_B>p_A)$ конверсии страницы Б выше А без учета и с учетом априорных данных. Априори ожидайте, что между группами нет разницы.

Для ответа вероятности переведите в проценты и округлите до целых, укажите вероятности без учета и с учетом априорных данных через пробел.

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b > samp_a) / npost

print("Without prior data:")
print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")
print()

N = 50000
Ns = 5200
alpha = Ns
beta = N - Ns

p_dist_a = stats.beta(a=alpha+sa, b=beta+na-sa)
p_dist_b = stats.beta(a=alpha+sb, b=beta+nb-sb)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b > samp_a) / npost

print("With prior data:")
print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")

#Without prior data:
#P(p_b > p_a) diff dist: 0.9894463764471134
#P(p_b > p_a) post samples: 0.98873
#
#With prior data:
#P(p_b > p_a) diff dist: 0.8276732342040869
#P(p_b > p_a) post samples: 0.82686

# Средние

### Модель

Для байесовской оценки параметров нормального распределения по выборке $x_i$ функция правдоподобия задается нормальным распределением $P(\mathcal{D} | \mathcal{H}) = \text{Norm}(x | \mu, \sigma_x^2)$. Используется упрощенная модель с одной случайной величиной $\mu$, значение $\sigma_x$ постоянно, хотя неизвестно и определяется из данных. Сопряженным априорным распределением будет нормальное распределение $P(\mu) = Norm(\mu | \mu_0, \sigma_0^2)$ с параметрами $\mu_0$, $\sigma_0$. Апостериорное распределение сохранит нормальную форму $P(\mu | \mathcal{D}) = Norm(\mu | \mu_N, \sigma_N^2)$ с обновленными параметрами $\mu_N$, $\sigma_N$.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Norm(x | \mu, \sigma_x^2) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\tfrac{(x - \mu)^2}{2 \sigma_x^2}}
\\
P(\mathcal{H}) & = Norm(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
\\
P(\mathcal{H} | \mathcal{D}) 
& = Norm(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{\sigma_x^2} \sum_i^N x_i
\end{split}
$$

### Задача, разность аналитически

Пусть в двух группах оценки апостериорных распределений $\mu$ получились $\mu_A \sim \text{Norm}(a,s_a^2)$ и $\mu_B \sim \text{Norm}(b,s_b^2)$. Посчитайте распределение разности $\mu_B − \mu_A$. В ответе запишите отношение среднего к корню из дисперсии.


Разность двух случайных величин с нормальным распределением будет распределена нормально.
Среднее равно разности средних, дисперсия - сумме дисперсий $\mu_B − \mu_A \sim \text{Norm}(b−a,s_a^2+s_b^2)$.
См. https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables .
В ответе ожидается `(b-a) / sqrt(s_a**2 + s_b**2)`.

# Транзакционная выручка на пользователя

### Модель

В выручке на пользователя $P_{пользователи}(x)$ удобно выделить выручку на платящего $P_{платящие}(x)$. При конверсии в оплату $p$ распределение ненулевой выручки на пользователя $p P_{платящие}(x)$, с вероятностью $1-p$ выручка нулевая.

$$
P_{пользователи}(x) = 
\begin{cases}
1-p, \, x = 0
\\
p P_{платящие}(x), \, x > 0
\end{cases}
$$

Для транзакционных сервисов, в частности маркетплейсов, выручку на платящего часто можно моделировать логнормальным распределением. Случайная величина логнормальная $X \sim \text{Lognormal}(\mu, s^2)$, если логарифм распределен нормально $\ln(X) \sim \text{Norm}(\mu, s^2)$.

Сопряженное априорное распределение к логнормальной функции правдоподобия $P(\mathcal{D} | \mathcal{H}) = \text{Lognorm}(x | \mu, s^2)$ строится аналогично нормальному распределению. Для упрощенной модели с одним параметром $\mu$ и фиксированным $s$ сопряженное априорное распределение нормальное $P(\mu) = \text{Norm}(\mu | \mu_0, \sigma_0^2)$ с параметрами $\mu_0$, $\sigma_0$. Апостериорное распределение нормальное $P(\mu | \mathcal{D}) = \text{Norm}(\mu | \mu_N, \sigma_N^2)$ с обновленными параметрами $\mu_N$, $\sigma_N$. В $\mu_N$ суммируются логарифмы точек выборки.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = \text{Lognorm}(x | \mu, s^2) = 
\frac{1}{x \sqrt{2 \pi s^2}} e^{-\tfrac{(\ln x - \mu)^2}{2 s^2}}
\\
P(\mathcal{H}) & = \text{Norm}(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
\\
P(\mathcal{H} | \mathcal{D}) 
& = \text{Norm}(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 s^2}{s^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{s^2} \sum_i^N \ln x_i
\end{split}
$$

Среднее логнормального распределения
$$
E[x] = e^{\mu + s^2/2}
$$

# Множественные сравнения

### Задача 1

В эксперименте с 3 группами на кажду версию веб-страницы зашло $N=10000$ человек. В версии $A$ кнопку "Продолжить" нажали 1030 человек, в $B$ - 1050, в $C$ - 1100. Посчитайте вероятность конверсии $C$ больше $A$ и $B$ одновременно $P(p_C > p_A \cap p_C > p_B)$. В ответе переведите вероятность в проценты и округлите до целых.


In [ ]:
import numpy as np
from scipy import stats

N = 10000
sa = 1030
sb = 1050
sc = 1100

p_dist_a = stats.beta(a=sa+1, b=N-sa+1)
p_dist_b = stats.beta(a=sb+1, b=N-sb+1)
p_dist_c = stats.beta(a=sc+1, b=N-sc+1)

npost = 500000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_c = p_dist_c.rvs(size=npost)

pc_gt_papb = np.sum((samp_c > samp_a) & (samp_c > samp_b)) / npost

print(f"P(p_c > p_a & p_c > p_b): {pc_gt_papb}")

#P(p_c > p_a & p_c > p_b): 0.842294

### Оценки вероятности пересечения

Вероятность пересечения нескольких событий $X_1, \ldots, X_n$ [меньше или равна вероятности отдельных событий](https://en.wikipedia.org/wiki/Fr%C3%A9chet_inequalities) $P(X_1 \cap \ldots \cap X_n) \le \min[P(X_1), \ldots, P(X_n)]$. Поэтому вероятность, что метрика в группе наибольшая, меньше или равна попарным вероятностям $P(p_A > p_B \cap p_A > p_C) \le \min[P(p_A > p_B), P(p_A > p_C)]$, то же для других групп.

При пороге $P(X_i) \ge 1 - p/n$ для каждого $X_i$ по соотношению $P(X_1 \cap \ldots \cap X_n) \ge P(X_1) + \ldots + P(X_n) - (n-1)$ вероятность пересечения будет больше $P(X_1 \cap \ldots \cap X_n) \ge 1 - p$. Такое условие на $P(X_i)$ называют [поправкой Бонферрони](https://en.wikipedia.org/wiki/Boole%27s_inequality#Example).

$$
\begin{split}
P(X_1 \cap \ldots \cap X_n) & = 1 - P\left((X_1 \cap \ldots \cap X_n)^c\right)
\\
& = 1 - P(X_1^c \cup \ldots \cup X_n^c)
\\
& \ge 1 - P(X_1^c) - \ldots - P(X_n^c)
\\
& \ge P(X_1) + \ldots + P(X_n) - (n-1)
\\
\end{split}
\\
\begin{split}
\text{Поправка Бонферрони: } & P(X_i) \ge 1 - \frac{p}{n} 
\\
& P(X_1 \cap \ldots \cap X_n) \ge 1 - p
\end{split}
$$

### Задача 2

Проверьте соотношение $P(X_1) + \ldots + P(X_n) - (n-1) \le P(X_1 \cap \ldots \cap X_n) \le \min[P(X_1), \ldots, P(X_n)]$.

В эксперименте с 3 группами на кажду версию веб-страницы зашло $N=10000$ человек. В версии $A$ кнопку "Продолжить" нажали 1030 человек, в $B$ - 1050, в $C$ - 1100. Посчитайте нижнюю границу $P(p_C > p_A \cap p_C > p_B)$ по соотношению $P(X_1 \cap \ldots \cap X_n) \ge P(X_1) + \ldots + P(X_n) - (n-1)$, вероятность $P(p_C > p_A \cap p_C > p_B)$ сравнением апостериорных распределений и меньшую из попарных вероятностей $\min[P(p_C > p_A), P(p_C > p_B)]$. В ответе укажите вероятности через пробел, переведите в проценты и округлите до целых: 70 75 85.

In [ ]:
import numpy as np
from scipy import stats

N = 10000
sa = 1030
sb = 1050
sc = 1100

p_dist_a = stats.beta(a=sa+1, b=N-sa+1)
p_dist_b = stats.beta(a=sb+1, b=N-sb+1)
p_dist_c = stats.beta(a=sc+1, b=N-sc+1)

npost = 500000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_c = p_dist_c.rvs(size=npost)

pc_gt_papb = np.sum((samp_c > samp_a) & (samp_c > samp_b)) / npost
pc_gt_pa = np.sum(samp_c > samp_a) / npost
pc_gt_pb = np.sum(samp_c > samp_b) / npost

print(f"P(p_c > p_a) + P(p_c > p_b) - 1: {pc_gt_pa + pc_gt_pb - 1}")
print(f"P(p_c > p_a & p_c > p_b): {pc_gt_papb}")
print(f"P(p_c > p_a): {pc_gt_pa}")
print(f"P(p_c > p_b): {pc_gt_pb}")

#P(p_c > p_a) + P(p_c > p_b) - 1: 0.819554
#P(p_c > p_a & p_c > p_b): 0.843242
#P(p_c > p_a): 0.946064
#P(p_c > p_b): 0.87349

# Связь с p-значениями

[Байесовские А/Б-тесты: связь с p-значениями](https://github.com/andrewbrdk/Bayesian-AB-Testing/blob/main/appendices/%D0%A1%D0%B2%D1%8F%D0%B7%D1%8C_%D1%81_p-%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%D0%BC%D0%B8.ipynb)

Показана численная близость p-значений t-теста, χ2-теста и U-критерия Манна-Уитни в А/Б-тестах вероятностям лучшей группы байесовских моделей. Соотношения выполняются несмотря на различия в определениях.

### Последовательное обновление

О Байесовском моделировании можно думать как об обновлении вероятностей гипотез по мере поступления новых данных.
Формулируются гипотезы $H_i$. Для них задаются априорные вероятности $P(H_i)$. По мере поступления данных $x_i$ для каждой гипотезы считается вероятность $P(H_i \cap x_1 \dots \cap x_N)$ - функции правдоподобия. Апостериорная вероятность $P(\mathcal{H} | \mathcal{D})$ будет равна вероятности получить данные в рамках гипотезы нормированной на вероятности в рамках всех гипотез

$$
P(\mathcal{H}_i | \mathcal{D}) = \frac{P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i)}{\sum_j P(\mathcal{D} | \mathcal{H}_j) P(\mathcal{H}_j)}
$$

<center>
<img src="../figs/bayes_update.png" alt="bayes_update" width="500"/>
<em>
<br/>
Обновление вероятностей гипотез по мере поступления данных.
</em>
</center>

### Задача про оценку среднего

Зафиксировать распределение выручки на пользователя как p * lognorm.  
Сгенерировать выборку.  
Сравнить среднюю выручку на пользователя по ЦПТ с точной. 

In [ ]:
def exact_rev_per_user_rvs(p, mu, s, nsamples):
    conv = stats.bernoulli.rvs(p=p, size=nsamples)
    rev = stats.lognorm.rvs(s=s, scale=np.exp(mu), size=nsamples)
    return conv * rev

#p = 
s = 1
mu = 8
nsample = 5000
exact_dist = stats.lognorm(s=s, scale=np.exp(mu))
data = exact_dist.rvs(nsample)

xaxis_max=30000
x = np.linspace(0, xaxis_max, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_dash='solid', line_color='black', name='Точное распределение'))
fig.add_trace(go.Scatter(x=[np.log(exact_dist.mean()), np.log(exact_dist.mean())], y=[0, max(exact_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', name='Логарифм точного среднего'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(exact_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', name='Точное среднее'))
fig.update_layout(title='$\mbox{Lognorm распределение } x$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()

Можно не усложнять все логнормальными распределениями.  
Попробовать просто распределение Бернулли * 5000, т.е. {0, 5000}. 

2 группы с распр. Бернулли.  
A: p, x,  
B: p \* 0.95, x \* 1.05

Средние:  
A: p \* x   
B: p \* x \* 0.95 \* 1.05  

Сгенерировать выборки.  
По выборкам посчитать относительную разность.  
Сравнить с реальной.  

In [ ]:
pa = 0.1
a = 5000
pb = pa * 0.95
b = a * 1.07



### Модель для нормального распределения с двумя параметрами

Два параметра: $\mu, \sigma$.  
Сопряженное априорное распределение - произведение нормального и гамма-распределений.


$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = P(x | \mu, \sigma_x) = \text{Norm}(x | \mu, \sigma_x)
\\
P(\mathcal{H}) & = \text{Norm}(\mu | \tau; \mu_0, k_0) \text{Gamma}(\tau; a, b)
\\
P(\mathcal{H} | \mathcal{D}) & =  \text{Norm}(\mu | \tau; \mu_N, k_N) \text{Gamma}(\tau; a_N, b_N)
\end{split}
\\
k_N = k_0 + N,
\qquad
\mu_N = \frac{k_0}{k_N}\mu_0 + \frac{1}{k_N}\sum_i^N x_i,
\qquad
a_N = a + N/2,
\qquad
b_N = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N x_i^2
\\
\text{Norm}(x | \mu, \sigma_x) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\tfrac{(x - \mu)^2}{2 \sigma_x^2}},
\qquad
\text{Gamma}(\tau; a, b) = \frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0
$$

Маржинальными распределениями будут гамма-распределение по $\tau$ и [нестандартизованное $t$-распределение](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Location-scale_t-distribution) по $\mu$: 
  

$$
P(\tau | \mathcal{D}) = \int d\mu P(\mu, \tau | \mathcal{D}) = \text{Gamma}(\tau; a_N, b_N)
$$

$$
\begin{split}
P(\mu | \mathcal{D}) & = \int d\tau P(\mu, \tau | \mathcal{D}) 
= 
t \left( \mu; \nu = 2 a_N, \mu_t = \mu_N, \tau= \left( \frac{b_N}{2 a_N k_N} \right)^{1/2} \right)
\end{split}
$$

Сопряженное априорное распределение - произведение нормального и гамма-распределений.

$$
P(model) = 
Norm(\mu | \tau; \mu_0, k_0) Gamma(\tau; a, b) = 
(\tau k_0/ \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$


$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
(\tau/ \pi)^{1/2} e^{- \tau (x-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_1 (\mu-\mu_1)^2}
\tau^{a_1-1} e^{-b_1 \tau}
\\
& = Norm(\mu | \tau; \mu_1, k_1) Gamma(\tau; a_1, b_1)
\end{split}
$$

$$
a_1 = a + 1/2
$$

$$
(x-\mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + 1) - 2 \mu (x + k_0 \mu_0) + x^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + 1) \left[ \mu^2 - 2 \mu \frac{x + k_0 \mu_0}{k_0 + 1} + \left(\frac{x + k_0 \mu_0}{k_0 + 1}\right)^2 \right]
- \frac{(x + k_0 \mu_0)^2}{k_0 + 1} + x^2 + k_0 \mu_0^2 + b
\\
= k_1 (\mu-\mu_1)^2 + b_1
$$

$$
k_1 = k_0 + 1
\\
\mu_1 = \frac{x + k_0 \mu_0}{k_0 + 1}
\\
b_1 = b + x^2 + k_0 \mu_0^2 - \frac{(x + k_0 \mu_0)^2}{k_0 + 1}
\\
= b + k_0 \mu_0^2 - k_1 \mu_1^2 + x^2
$$

$$
b_1 = b + \frac{x^2 k_0 + x^2 + k_0^2 \mu_0^2 + k_0 \mu_0^2 - x^2 - 2xk_0\mu_0 - k_0^2 \mu_0^2}{k_0+1}
= b + \frac{x^2 k_0 + k_0 \mu_0^2 - 2xk_0\mu_0}{k_0+1}
\\
= b + \frac{k_0}{k_0+1}(x - \mu_0)^2
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
\prod_i^N (\tau/ \pi)^{1/2} e^{- \tau (x_i-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& = Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\end{split}
$$

$$
a_N = a + N/2
$$

$$
\sum_i^N (x_i - \mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + N) - 2 \mu (k_0 \mu_0 + \sum_i^N x_i) + \sum_i^N x_i^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + N) \left[ \mu^2 - 2 \mu \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N} + 
\left(\frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}\right)^2 \right]
- \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N} + \sum_i^N x_i^2 + k_0 \mu_0^2 + b
\\
= k_N (\mu-\mu_N)^2 + b_N
$$

$$
k_N = k_0 + N
\\
\mu_N = \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}
\\
b_N = b + \sum_i^N x_i^2 + k_0 \mu_0^2 - \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N}
\\
b_N = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N x_i^2
$$

Маржинальные распределения

По $\tau$ гамма-распределение

$$
P(\tau | data) = Gamma(\tau; a_N, b_N)
$$

По $\mu$ $t$-распределение  
https://en.wikipedia.org/wiki/Student%27s_t-distribution#Location-scale_t_distribution  

$$
\begin{split}
P(\mu | data) & = \int d\tau P(\mu, \tau | data)
\\
& = \int d\tau Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\\
& \propto_{\mu}
\int d\tau 
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& \propto_{\mu}
\int d\tau
\tau^{a_N + 1/2 -1} e^{-\tau (b_N + k_N (\mu-\mu_N)^2)}
\\
& \propto_{\mu}
\frac{\Gamma(a_N + 1/2)}{(b_N + k_N (\mu-\mu_N)^2)^{a_N + 1/2}}
\\
& \propto_{\mu}
\left(1 + \frac{1}{2 a_N} \frac{2 a_N k_N}{b_N} (\mu-\mu_N)^2\right)^{-(2 a_N + 1)/2}
\\
& = lst \left( \mu; \nu = 2 a_N, \mu_t = \mu_N, \tau= \left( \frac{b_N}{2 a_N k_N} \right)^{1/2} \right)
\end{split}
$$

### Стат. значимость, мощность.

Похожую ситуацию рассматривают в "проверках нулевых гипотез". Вместо улучшения и ухудшения формулируют гипотезу $H_0$.

<center>
<img src="../figs/bayes_square_h0.png" alt="bayes_square_h0" width="500"/>
</center>

$$
P(\text{правильный выбор}) = h (1-a) + (1-h) b
$$

Обратите внимание, что вероятность "правильного выбора" зависит от качества предлагаемых изменений. 
Если изменения чаще улучшают продукт, она будет ближе к $1-a$, если ухудшают - к $b$.

In [ ]:
N = 50
a = 0.05
#b = 0.2
b = a
h = 0.2
S = N * h * (1-a) / (h * (1-a) + (1-h)*b)
print('S', S)

S = N * h * (1-a)
print('S', S)

L = N * (1-h) * (1-b)
print('L', L)